# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124179e+02     1.655843e+00
 * time: 0.12420296669006348
     1     1.106896e+01     1.042905e+00
 * time: 0.9522190093994141
     2    -1.231032e+01     1.090162e+00
 * time: 1.0885429382324219
     3    -3.431624e+01     8.043889e-01
 * time: 1.2412199974060059
     4    -4.780720e+01     6.372776e-01
 * time: 1.38714599609375
     5    -5.693492e+01     2.052731e-01
 * time: 1.5241990089416504
     6    -5.980029e+01     1.318435e-01
 * time: 1.6367950439453125
     7    -6.092592e+01     5.419014e-02
 * time: 1.749932050704956
     8    -6.137286e+01     6.437799e-02
 * time: 1.857593059539795
     9    -6.167170e+01     4.260125e-02
 * time: 1.9724760055541992
    10    -6.189294e+01     2.681866e-02
 * time: 2.086672067642212
    11    -6.204535e+01     1.834371e-02
 * time: 2.1961429119110107
    12    -6.210175e+01     1.566181e-02
 * time: 2.311552047729492
    13    -6.215143e+01     1.132120e-02
 * time: 2.432185888290405

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671087
    AtomicLocal         -18.8557692
    AtomicNonlocal      14.8522630
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336829

    total               -62.261666459154
